# How to use the RNN Autoencoder with NILMTK

This is an example on how to train and use the Recurrent Network (RNN) disaggregator on the [REDD](http://redd.csail.mit.edu/) dataset using [NILMTK](https://github.com/nilmtk/NILMTK/).

This network was described in the [Neural NILM](https://arxiv.org/pdf/1507.06594.pdf) paper.

First of all, we need to train the RNNDisaggregator using the train data. For this example, both train and test data are consumption data of the microwave of the first REDD building.

In [2]:
import warnings; warnings.simplefilter('ignore')

from nilmtk import DataSet
train = DataSet(r'C:\Users\micki\nilmtk_test\data\converted_v14.h5') #new version of train file 
train.set_window(end="2021-10-02") #because 2021-10-06 starts house1 
# from test19 file 
# TIMESTAMP_COLUMN_NAME = "timestamp"
# TIMEZONE = "Europe/London"
# START_DATETIME, END_DATETIME = '2021-09-30', '2021-10-05'
train_elec = train.buildings[1].elec

Next, we need to define the disaggregator model.

In [3]:
from grudisaggregator import GRUDisaggregator
gru = GRUDisaggregator()

In [4]:
from nilmtk import DataSet

sel = DataSet(r'C:\Users\micki\nilmtk_test\data\converted_v14.h5')
elec = sel.buildings[1].elec
elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='SEL', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='SEL', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=3, building=1, dataset='SEL', appliances=[Appliance(type='electric oven', instance=1)])
  ElecMeter(instance=4, building=1, dataset='SEL', appliances=[Appliance(type='kettle', instance=1)])
  ElecMeter(instance=5, building=1, dataset='SEL', appliances=[Appliance(type='stove', instance=1)])
)

In [5]:
#

Then train the model. We need to input the train data as well as their sample period. Also, we need to pass the desired number of training epochs. Finally, save the model for later use.

In [6]:
train_mains = train_elec.mains() # The aggregated meter that provides the input
train_meter = train_elec.submeters()['fridge'] # The microwave meter that is used as a training target

gru.train(train_mains, train_meter, epochs=5, sample_period=1)
gru.export_model("model-sel_v05.h5")

Epoch 1/5
1322/1322 [==============================] - 9s 7ms/step - loss: 2.4418e-05
Epoch 2/5
1322/1322 [==============================] - 9s 7ms/step - loss: 2.3813e-05
Epoch 3/5
1322/1322 [==============================] - 9s 6ms/step - loss: 2.3714e-05
Epoch 4/5
1322/1322 [==============================] - 9s 7ms/step - loss: 2.3574e-05
Epoch 5/5
1322/1322 [==============================] - 10s 8ms/step - loss: 2.3556e-05


Now that the model is trained, we can use it to disaggregate energy data. Let's test it on the rest of the data from building 1.

First we use the model to predict the microwave consumption. The results are saved automatically in a .h5 datastore.

In [7]:
test = DataSet(r'C:\Users\micki\nilmtk_test\data\converted_v14.h5')
test.set_window(start="2021-09-30", end="2021-10-05")
test_elec = test.buildings[1].elec
test_mains = test_elec.mains()

disag_filename = 'disag-outSEL_v05.h5' # The filename of the resulting datastore
from nilmtk.datastore import HDFDataStore
output = HDFDataStore(disag_filename, 'w') # here you change to 'a' or 'w' depending on case 

# test_mains: The aggregated signal meter
# output: The output datastore
# train_meter: This is used in order to copy the metadata of the train meter into the datastore
gru.disaggregate(test_mains, output, train_meter, sample_period=1)

New sensible chunk: 428401


Let's plot the results and compare them to the ground truth signal.

**Note:** Calling plot this way, downsamples the signal to reduce computing time. To plot the entire signal call
```
predicted.power_series_all_data().plot()
ground_truth.power_series_all_data().plot()
```

In [15]:

result = DataSet(disag_filename)
res_elec = result.buildings[1].elec
predicted = res_elec['fridge']
ground_truth = test_elec['fridge']

import matplotlib.pyplot as plt
predicted.plot()
ground_truth.plot()
plt.show()

OSError: HDF5 error back trace

  File "..\src\H5F.c", line 509, in H5Fopen
    unable to open file
  File "..\src\H5Fint.c", line 1652, in H5F_open
    unable to read superblock
  File "..\src\H5Fsuper.c", line 632, in H5F__super_read
    truncated file: eof = 96, sblock->base_addr = 0, stored_eof = 2048

End of HDF5 error back trace

Unable to open/create file 'disag-out2.h5'

Finally let's see the metric results.

In [19]:
import metrics
rpaf = metrics.recall_precision_accuracy_f1(predicted, ground_truth)
print("============ Recall: {}".format(rpaf[0]))
print("============ Precision: {}".format(rpaf[1]))
print("============ Accuracy: {}".format(rpaf[2]))
print("============ F1 Score: {}".format(rpaf[3]))

print("============ Relative error in total energy: {}".format(metrics.relative_error_total_energy(predicted, ground_truth)))
print("============ Mean absolute error(in Watts): {}".format(metrics.mean_absolute_error(predicted, ground_truth)))

NameError: name 'predicted' is not defined